**Transfer Learning com cats_vs_dogs no Google Colab**
Este notebook demonstra o processo de Transfer Learning para classificação de imagens usando um modelo pré-treinado (MobileNetV2) e o dataset cats_vs_dogs.

**Passo 1: Configuração do Ambiente e Importações**
Primeiro, importamos todas as bibliotecas necessárias.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import BinaryCrossentropy

**Passo 2: Carregamento e Preparação do Dataset** cats_vs_dogs

Aqui, usamos o tensorflow_datasets para carregar o dataset. Dividimos o conjunto de treinamento original em 80% para treino e 20% para validação. as_supervised=True garante que o dataset retorne tuplas (imagem, rótulo).

In [2]:
# Carrega o dataset e as informações sobre ele
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print("Dataset de treino original:", raw_train)
print("Dataset de validação original:", raw_validation)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.1Y4FBD_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Dataset de treino original: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
Dataset de validação original: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


**Passo 3: Pré-processamento dos Dados**

O modelo MobileNetV2 espera imagens de um tamanho específico (usaremos 160x160) e com pixels normalizados no intervalo [-1, 1]. Criamos uma função para aplicar essas transformações.

In [3]:
IMG_SIZE = (160, 160)

def format_example(image, label):
  """
  Redimensiona a imagem para o tamanho esperado e normaliza os pixels.
  """
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1 # Normaliza para o intervalo [-1, 1]
  image = tf.image.resize(image, IMG_SIZE)
  return image, label

# Aplica a função de formatação aos datasets
train_dataset = raw_train.map(format_example)
validation_dataset = raw_validation.map(format_example)
test_dataset = raw_test.map(format_example)

Para melhorar a performance do treinamento, usamos shuffle, batch e prefetch.

In [4]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_batches = validation_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_batches = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Vamos dar uma olhada em um batch de dados
for image_batch, label_batch in train_batches.take(1):
   pass

print("Shape do batch de imagens:", image_batch.shape)
print("Shape do batch de rótulos:", label_batch.shape)

Shape do batch de imagens: (32, 160, 160, 3)
Shape do batch de rótulos: (32,)


**Passo 4: Construção do Modelo (Feature Extraction)**

Carregamos o MobileNetV2 pré-treinado no ImageNet, sem sua camada de classificação (include_top=False). Em seguida, "congelamos" os pesos do modelo base para que eles não sejam atualizados durante o treinamento inicial.

In [5]:
# Define o shape de entrada
IMG_SHAPE = IMG_SIZE + (3,)

# Carrega o modelo base pré-treinado
base_model = MobileNetV2(input_shape=IMG_SHAPE,
                         include_top=False,
                         weights='imagenet')

# Congela o modelo base
base_model.trainable = False
base_model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "mobilenetv2_1.00_160"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

Agora, adicionamos nossa própria camada de classificação no topo do modelo base. Usamos GlobalAveragePooling2D para converter os mapas de características em um único vetor por imagem.

In [6]:
# Adiciona as novas camadas de classificação
inputs = Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False) # training=False é importante aqui!
x = GlobalAveragePooling2D()(x)
# A camada final tem 1 neurônio, pois é uma classificação binária (gato ou cão)
outputs = Dense(1)(x)

model = Model(inputs, outputs)

**Passo 5: Compilação e Treinamento Inicial**

Compilamos o modelo usando Adam como otimizador e BinaryCrossentropy como função de perda, ideal para classificação binária.

In [7]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Treinamos o modelo por algumas épocas apenas com as novas camadas.

In [8]:
initial_epochs = 3
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

Epoch 1/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 390s 657ms/step - accuracy: 0.8229 - loss: 0.3689 - val_accuracy: 0.9630 - val_loss: 0.1172
Epoch 2/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 383s 656ms/step - accuracy: 0.9698 - loss: 0.0991 - val_accuracy: 0.9725 - val_loss: 0.0815
Epoch 3/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 423s 724ms/step - accuracy: 0.9776 - loss: 0.0707 - val_accuracy: 0.9776 - val_loss: 0.0690


**Passo 6: Ajuste Fino (Fine-Tuning)**

Após o treinamento inicial, podemos "descongelar" as camadas superiores do modelo base e continuar o treinamento com uma taxa de aprendizado (learning_rate) muito baixa. Isso ajustará os pesos das características mais específicas para o nosso dataset.

In [9]:
# Descongela o modelo base
base_model.trainable = True

# Vamos descongelar a partir da camada 100
fine_tune_at = 100

# Congela todas as camadas antes da 'fine_tune_at'
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Re-compila o modelo com uma taxa de aprendizado muito mais baixa
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,862,721 (7.11 MB)

 Non-trainable params: 396,544 (1.51 MB)

Continuamos o treinamento.

In [10]:
fine_tune_epochs = 3
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_batches)

Epoch 3/6
582/582 ━━━━━━━━━━━━━━━━━━━━ 635s 1s/step - accuracy: 0.9281 - loss: 0.2014 - val_accuracy: 0.9798 - val_loss: 0.0714
Epoch 4/6
582/582 ━━━━━━━━━━━━━━━━━━━━ 664s 1s/step - accuracy: 0.9701 - loss: 0.0739 - val_accuracy: 0.9811 - val_loss: 0.0537
Epoch 5/6
582/582 ━━━━━━━━━━━━━━━━━━━━ 625s 1s/step - accuracy: 0.9785 - loss: 0.0548 - val_accuracy: 0.9802 - val_loss: 0.0529
Epoch 6/6
582/582 ━━━━━━━━━━━━━━━━━━━━ 660s 1s/step - accuracy: 0.9832 - loss: 0.0432 - val_accuracy: 0.9785 - val_loss: 0.0542


**Passo 7: Avaliação Final**

Finalmente, avaliamos o modelo no conjunto de teste para ver sua performance em dados nunca vistos.

In [11]:
loss, accuracy = model.evaluate(test_batches)
print('Acurácia no conjunto de teste:', accuracy)

73/73 ━━━━━━━━━━━━━━━━━━━━ 43s 578ms/step - accuracy: 0.9833 - loss: 0.0441
Acurácia no conjunto de teste: 0.979363739490509


**Passo 8: Realizando a Predição em uma Nova Imagem da Internet**

Esta função encapsula todo o processo necessário:

Baixar a imagem a partir de um URL.
Carregá-la em um formato que o TensorFlow entenda.
Aplicar exatamente o mesmo pré-processamento que usamos para os dados de treino.
Realizar a predição.
Interpretar o resultado para o usuário.

In [12]:
import requests
from PIL import Image
import io

# Assegure que as outras bibliotecas já foram importadas
# import tensorflow as tf
# import numpy as np

def predict_image_from_url(image_url, model, img_size=(160, 160)):
    """
    Baixa uma imagem de um URL, a pré-processa e retorna a predição do modelo.

    Args:
        image_url (str): O link (URL) da imagem a ser classificada.
        model (tf.keras.Model): O modelo treinado.
        img_size (tuple): O tamanho para o qual a imagem deve ser redimensionada.
                          Deve ser o mesmo usado no treinamento.

    Returns:
        None. Imprime o resultado da classificação.
    """
    try:
        # 1. Baixar a imagem
        response = requests.get(image_url)
        # Garante que o request foi bem-sucedido
        response.raise_for_status()

        # 2. Carregar a imagem
        image_bytes = io.BytesIO(response.content)
        img = Image.open(image_bytes).convert('RGB') # Garante que a imagem tenha 3 canais (RGB)

        # 3. Pré-processar a imagem
        # Converte a imagem PIL para um tensor do TensorFlow
        img_tensor = tf.keras.preprocessing.image.img_to_array(img)

        # Aplica as mesmas transformações do treinamento
        img_tensor = tf.image.resize(img_tensor, img_size)
        img_tensor = (img_tensor / 127.5) - 1 # Normaliza para [-1, 1]

        # Adiciona uma dimensão de batch (o modelo espera um lote de imagens)
        # O shape passa de (160, 160, 3) para (1, 160, 160, 3)
        img_tensor = tf.expand_dims(img_tensor, axis=0)

        # 4. Realizar a predição
        prediction = model.predict(img_tensor)

        # 5. Interpretar o resultado
        # A saída do nosso modelo é um logit. Valores > 0 indicam uma classe, < 0 indicam a outra.
        # Aplicamos a função sigmoide para converter o logit em uma "probabilidade" entre 0 e 1.
        score = tf.nn.sigmoid(prediction[0][0])

        confidence = 100 * (1 - score if score < 0.5 else score)

        # O tfds.load('cats_vs_dogs') define 'gato' como classe 0 e 'cachorro' como 1.
        # Portanto, um score < 0.5 indica gato, e > 0.5 indica cachorro.
        if score < 0.5:
            print(f"🐾 A imagem é um GATO com {confidence:.2f}% de confiança.")
        else:
            print(f"🐶 A imagem é um CACHORRO com {confidence:.2f}% de confiança.")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar a imagem: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

**Teste com uma imagem sua!**

In [21]:
# Encontre uma imagem de gato ou cachorro na internet, copie o link e cole aqui
my_image_url = "INSIRA A URL DA SUA IMAGEM"
predict_image_from_url(my_image_url, model)

Erro ao baixar a imagem: Invalid URL 'INSIRA A URL DA SUA IMAGEM': No scheme supplied. Perhaps you meant https://INSIRA A URL DA SUA IMAGEM?
